In [45]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as Dataset
import torchvision.transforms.functional as TVF
import torch.autograd as autograd
from torch.utils.data import Dataset, DataLoader

from torchvision.models import inception_v3
from PIL import Image

import nltk
from gensim.models import Word2Vec

In [76]:
columns = ['filename', 'caption']
caption_data = pd.read_csv('./Flickr8k_text/Flickr8k.token.txt' , 
                           sep = '\t' ,
                          header = None,
                          names = columns)
caption_data.head()

,filename,caption
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


In [92]:
caption_data['caption_num'] = caption_data['filename'].str.split('#',expand=True)[1]
caption_data.head()

,filename,caption,caption_num
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...,0
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .,1
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .,2
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...,3
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...,4


In [93]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size = 1024):
        super(EncoderCNN, self).__init__()
        
        # get the pretrained densenet model
        self.inception_v3 = inception_v3(pretrained=True)
        
    def forward(self, x):
        
        #get embedding from inception
        x = self.inception_v3(x) 
        
        return x

In [94]:
image = Image.open('.\\Flickr8k_Dataset\\Flicker8k_Dataset\\667626_18933d713e.jpg')

In [95]:
x = TVF.to_tensor(image)
x.unsqueeze_(0)
print(x.shape)

torch.Size([1, 3, 375, 500])


In [96]:
encoder_rnn = EncoderCNN()